In [33]:
 # detect and track faces
# %load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [66]:
import argparse
import os
import pickle
import sys
from pathlib import Path

import numpy as np
from PIL import Image
from moviepy.editor import VideoFileClip
from tqdm import tqdm

from tqdm import tqdm_notebook

In [39]:
# add project to python path
sys.path.append('../../')
from src.align.align_trans import get_reference_facial_points, warp_and_crop_face
from src.align.detector import detect_faces
from src.align.get_nets import PNet, RNet, ONet

set variables and Paths

In [56]:
source_root="../data/fiw-mm/"
dest_root="../data/fiw-mm/interm/visual/video-frame-faces/"
crop_size=112
fps = 25

path_data = Path(source_root).resolve()
path_images = path_data / 'FIDs-MM/visual/image'
path_videos = path_data / 'FIDs-MM/visual/video'
path_encodings = path_data / 'features/image/arcface'
path_out = Path(dest_root).resolve()
path_out.mkdir(exist_ok=True, parents=True)

In [65]:
%%capture
pnet = PNet()
rnet = RNet()
onet = ONet()

In [ ]:
def unique_path(directory, name_pattern):
    counter = -1
    while True:
        counter += 1
        path = directory / name_pattern.format(counter)
        if not path.exists():
            return path

set / get parameters and get files for face detector and to process, respectively

In [ ]:
scale = crop_size / 112.
reference = get_reference_facial_points(default_square=True) * scale

f_videos = path_videos.rglob('*.mp4')

# for f_video in tqdm(f_videos):
for f_video in tqdm_notebook(f_videos):
    ref_base = str(f_video).replace(f"{str(path_videos)}/", "")
    path_obin = (path_out / ref_base).with_suffix('')
    try:
        path_obin.mkdir(parents=True)
    except:
        print('skipping', f_video)
        continue
    print("Processing\t{}".format(ref_base))
    clip = VideoFileClip(str(f_video))
    tracks = []
    for k, frame in enumerate(clip.iter_frames(fps=fps)):
        bbs, landmarks = detect_faces(Image.fromarray(frame), pnet=pnet, rnet=rnet, onet=onet)
        ndetections = len(landmarks)
        path_image_out = path_obin / 'frame-{:03d}.jpg'.format(k)
        if ndetections:  # If the landmarks cannot be detected, the img will be discarded
            for i in range(ndetections):
                facial5points = [[landmarks[i][j], landmarks[i][j + 5]] for j in range(5)]
                warped_face = warp_and_crop_face(frame, facial5points,reference, crop_size=(crop_size, crop_size))

                path_image_out = path_obin / str(Path(path_image_out).name).replace('.jpg', '-{:02d}.jpg'.format(i))                                  
                img_warped = Image.fromarray(warped_face)
                img_warped.save(str(path_image_out))
                                                                  
                path_lmarks_out = str(path_image_out).replace('.jpg', '-landmarks.csv')
                path_bb_out = str(path_image_out).replace('.jpg', '-bb.csv')

                np.savetxt(str(path_lmarks_out), landmarks[i], delimiter=',')
                np.savetxt(path_bb_out, bbs[i], delimiter=',')
        else:
            print("{} is discarded due to non-detected landmarks!".format(path_image_out))


skipping /Volumes/MyWorld/FIW-MM/data/FIDs-MM/visual/video/F0005/MID3/76dkmAvXHPE_0000001.mp4
skipping /Volumes/MyWorld/FIW-MM/data/FIDs-MM/visual/video/F0005/MID3/76dkmAvXHPE_0000002.mp4
skipping /Volumes/MyWorld/FIW-MM/data/FIDs-MM/visual/video/F0005/MID3/DOsdKD-VRrA_0000001.mp4
skipping /Volumes/MyWorld/FIW-MM/data/FIDs-MM/visual/video/F0005/MID3/DOsdKD-VRrA_0000002.mp4
skipping /Volumes/MyWorld/FIW-MM/data/FIDs-MM/visual/video/F0005/MID3/DOsdKD-VRrA_0000003.mp4
skipping /Volumes/MyWorld/FIW-MM/data/FIDs-MM/visual/video/F0005/MID3/EQLAtB1Kse4_0000001.mp4
skipping /Volumes/MyWorld/FIW-MM/data/FIDs-MM/visual/video/F0005/MID3/EQLAtB1Kse4_0000002.mp4
skipping /Volumes/MyWorld/FIW-MM/data/FIDs-MM/visual/video/F0005/MID3/EQLAtB1Kse4_0000003.mp4
skipping /Volumes/MyWorld/FIW-MM/data/FIDs-MM/visual/video/F0005/MID3/EQLAtB1Kse4_0000004.mp4
skipping /Volumes/MyWorld/FIW-MM/data/FIDs-MM/visual/video/F0005/MID3/EQLAtB1Kse4_0000005.mp4
skipping /Volumes/MyWorld/FIW-MM/data/FIDs-MM/visual/video/F

In [94]:
!open /Volumes/MyWorld/FIW-MM/data/FIDs-MM/visual/video/F0005/MID3/gudJriJgLQ4_0000001.mp4

In [58]:
ref_base = str(f_video).replace(f"{str(path_data)}/", "")
path_obin = (path_out / ref_base).with_suffix('')
path_obin
print(path_obin)

PosixPath('/Volumes/MyWorld/FIW-MM/data/interm/visual/video-frame-faces/FIDs-MM/visual/video/F0005/MID3/76dkmAvXHPE_0000001')

/Volumes/MyWorld/FIW-MM/data/interm/visual/video-frame-faces/FIDs-MM/visual/video/F0005/MID3/76dkmAvXHPE_0000001


In [63]:
print((landmarks[0]))

[ 31.645689  57.47556   55.507782  40.95931   60.545403 155.9029
 152.5538   166.5087   184.61024  181.15593 ]


In [62]:
?np.savetxt

In [ ]:

meta[ref] = li_meta
with open(source_root + 'cropped-meta.pkl', 'wb') as f:
    pickle.dump(meta, f)